In [2]:
import json
import os
from collections import namedtuple
import scipy.sparse
from sklearn.preprocessing import StandardScaler
import dgl
import numpy as np
import torch
from sklearn.metrics import f1_score

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.utils as u
import torch.autograd.profiler as profiler

In [3]:
args = None
multilabel = True

In [4]:
if not os.path.exists('graphsaintdata') and not os.path.exists('data'):
    raise ValueError("The directory graphsaintdata does not exist!")
elif os.path.exists('graphsaintdata') and not os.path.exists('data'):
    os.rename('graphsaintdata', 'data')
# prefix = "data/{}".format(args.dataset)#################################
prefix = "data/ppi"#################################
DataType = namedtuple('Dataset', ['num_classes', 'train_nid', 'g'])

In [5]:
adj_full = scipy.sparse.load_npz('./{}/adj_full.npz'.format(prefix)).astype(bool)#################
g = dgl.from_scipy(adj_full)
num_nodes = g.num_nodes()

In [6]:
adj_train = scipy.sparse.load_npz('./{}/adj_train.npz'.format(prefix)).astype(bool)#############
train_nid = np.array(list(set(adj_train.nonzero()[0])))

In [7]:
role = json.load(open('./{}/role.json'.format(prefix)))
mask = np.zeros((num_nodes,), dtype=bool)
train_mask = mask.copy()
train_mask[role['tr']] = True
val_mask = mask.copy()
val_mask[role['va']] = True
test_mask = mask.copy()
test_mask[role['te']] = True

In [8]:
feats = np.load('./{}/feats.npy'.format(prefix))
scaler = StandardScaler()
scaler.fit(feats[train_nid])
feats = scaler.transform(feats)

In [9]:
class_map = json.load(open('./{}/class_map.json'.format(prefix)))
class_map = {int(k): v for k, v in class_map.items()}

In [10]:
if multilabel:
    # Multi-label binary classification
    num_classes = len(list(class_map.values())[0])
    class_arr = np.zeros((num_nodes, num_classes))
    for k, v in class_map.items():
        class_arr[k] = v
else:
    num_classes = max(class_map.values()) - min(class_map.values()) + 1
    class_arr = np.zeros((num_nodes,))
    for k, v in class_map.items():
        class_arr[k] = v

In [11]:
g.ndata['feat'] = torch.tensor(feats, dtype=torch.float)
g.ndata['label'] = torch.tensor(class_arr, dtype=torch.float if multilabel else torch.long)
g.ndata['train_mask'] = torch.tensor(train_mask, dtype=torch.bool)
g.ndata['val_mask'] = torch.tensor(val_mask, dtype=torch.bool)
g.ndata['test_mask'] = torch.tensor(test_mask, dtype=torch.bool)

# Mapping
graphsaint -> mh-aug

In [12]:
features = g.ndata['feat']
labels = g.ndata['label']
num_classes = num_classes
bn = False

In [48]:
edge_index = torch.Tensor(adj_full.nonzero()).to(torch.int)
train_index = g.ndata['train_mask'].tolist()
train_label = labels[train_index]
valid_index = g.ndata['val_mask'].tolist()
valid_label = labels[valid_index]
test_index = g.ndata['test_mask'].tolist()
test_label = labels[test_index]

C:\Users\Laurelwoods\AppData\Local\Temp\ipykernel_22220\843409145.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  edge_index = torch.Tensor(adj_full.nonzero())


In [16]:
edge_index

(array([    0,     0,     0, ..., 14754, 14754, 14754]),
 array([  158,   160,   571, ..., 14431, 14616, 14675]))